In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Burgers 1D simplificada como proxy para NS behavior (desenvolve choque)
def burgers(t, u, nu, alpha, use_limiter):
    u = u.reshape((N,))
    ux = np.gradient(u, dx)
    uxx = np.gradient(ux, dx)
    
    adv = -u * ux
    visc = nu * uxx
    
    if use_limiter:
        # Medida simples de coerência: baixa vorticidade normalizada = alta coerência forçada
        sigma = np.mean(np.abs(ux)) / (np.mean(np.abs(u)) / L + 1e-8)  # evita divisão zero
        phi = np.tanh(beta * (sigma - sigma_c))  # ativa acima threshold
        Re_eff = np.mean(np.abs(u)) * L / nu
        f_rest = -alpha * phi / Re_eff * u  # puxa para repouso/multiflux aproximado
    else:
        f_rest = 0.0
    
    return (adv + visc + f_rest).flatten()

# Parâmetros
N = 256
L = 2*np.pi
dx = L / N
x = np.linspace(0, L, N, endpoint=False)

# Condição inicial: onda que tende a choque
u0 = np.sin(2*np.pi*x/L)**2 + 0.1*np.cos(4*np.pi*x/L)

nu = 0.01          # viscosidade baixa → tende a blowup-like
alpha = 5.0        # força do limitador
beta = 10.0        # steepness da ativação
sigma_c = 0.5      # threshold de coerência

# Simulação sem limitador
sol_no = solve_ivp(burgers, [0, 2.0], u0.flatten(), args=(nu, alpha, False),
                   method='RK45', rtol=1e-6)

# Simulação com limitador
sol_yes = solve_ivp(burgers, [0, 2.0], u0.flatten(), args=(nu, alpha, True),
                    method='RK45', rtol=1e-6)

# Plots comparativos
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.plot(x, sol_no.y[:,-1], label='Sem limitador')
plt.plot(x, sol_yes.y[:,-1], label='Com limitador')
plt.title('Perfil de velocidade final')
plt.legend()

plt.subplot(2,2,2)
energy_no = np.trapz(sol_no.y**2, x, axis=0)
energy_yes = np.trapz(sol_yes.y**2, x, axis=0)
plt.plot(sol_no.t, energy_no, label='Sem')
plt.plot(sol_yes.t, energy_yes, label='Com')
plt.title('Energia cinética')
plt.legend()

plt.subplot(2,2,3)
max_grad_no = np.max(np.abs(np.gradient(sol_no.y, dx, axis=0)), axis=0)
max_grad_yes = np.max(np.abs(np.gradient(sol_yes.y, dx, axis=0)), axis=0)
plt.plot(sol_no.t, max_grad_no, label='Sem')
plt.plot(sol_yes.t, max_grad_yes, label='Com')
plt.title('Gradiente máximo (proxy blowup)')
plt.legend()

plt.tight_layout()
plt.savefig('comparacao_limitador.png')
plt.show()